In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bids

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c92092d4f7f00524a7026e3dc73dddfa447bb6363b8fe27218eaebd814d4c1d3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
import os
import pandas as pd
import librosa
import librosa.display
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import vgg16
from collections import namedtuple
import requests
from tqdm import tqdm
import albumentations
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import utils as vutils

workdir = './drive/MyDrive/MRIVQGEN'
data_path = os.path.join(workdir, 'ds004814')
if not os.path.isdir(data_path):
  %mkdir -p './drive/MyDrive/MRIVQGEN/ds004814'

In [4]:
participant_metadata = pd.read_csv(os.path.join(data_path, 'participants.tsv'), sep='\t')
participant_metadata

,participant_id
0,sub-01
1,sub-02
2,sub-03
3,sub-04
4,sub-05
5,sub-06


In [5]:
from bids.layout import BIDSLayout

layout = BIDSLayout(data_path)

print(layout.get_tasks())

['auditory']


In [15]:
import nibabel as nib
import numpy as np

files = []
files = layout.get_files()

print(files)

{'/content/drive/MyDrive/MRIVQGEN/ds004814/README': <BIDSFile filename='/content/drive/MyDrive/MRIVQGEN/ds004814/README'>, '/content/drive/MyDrive/MRIVQGEN/ds004814/CHANGES': <BIDSFile filename='/content/drive/MyDrive/MRIVQGEN/ds004814/CHANGES'>, '/content/drive/MyDrive/MRIVQGEN/ds004814/dataset_description.json': <BIDSJSONFile filename='/content/drive/MyDrive/MRIVQGEN/ds004814/dataset_description.json'>, '/content/drive/MyDrive/MRIVQGEN/ds004814/participants.tsv': <BIDSDataFile filename='/content/drive/MyDrive/MRIVQGEN/ds004814/participants.tsv'>, '/content/drive/MyDrive/MRIVQGEN/ds004814/sub-03/ses-03/anat/sub-03_ses-03_T1w.json': <BIDSJSONFile filename='/content/drive/MyDrive/MRIVQGEN/ds004814/sub-03/ses-03/anat/sub-03_ses-03_T1w.json'>, '/content/drive/MyDrive/MRIVQGEN/ds004814/sub-03/ses-03/anat/sub-03_ses-03_T1w.nii.gz': <BIDSImageFile filename='/content/drive/MyDrive/MRIVQGEN/ds004814/sub-03/ses-03/anat/sub-03_ses-03_T1w.nii.gz'>, '/content/drive/MyDrive/MRIVQGEN/ds004814/sub-03

In [24]:
file = files.popitem()[1]
print(file.path)

layout.get_metadata(file.path)

/content/drive/MyDrive/MRIVQGEN/ds004814/sub-05/ses-01/anat/sub-05_ses-01_PDw.nii.gz


{'AcquisitionMatrixPE': 384,
 'AcquisitionNumber': 1,
 'AcquisitionTime': '12:17:0.080000',
 'BaseResolution': 384,
 'CoilCombinationMethod': 'Sum of Squares',
 'CoilString': 'C:O32',
 'ConversionSoftware': 'dcm2niix',
 'ConversionSoftwareVersion': 'v1.0.20230411',
 'DeviceSerialNumber': '18912',
 'DwellTime': 6.5e-06,
 'EchoTime': 0.00345,
 'FlipAngle': 4,
 'ImageOrientationPatientDICOM': [0, 1, 0, 0, 0, -1],
 'ImageOrientationText': 'Sag',
 'ImageType': ['ORIGINAL', 'PRIMARY', 'M', 'ND'],
 'ImagingFrequency': 296.609776,
 'InPlanePhaseEncodingDirectionDICOM': 'ROW',
 'InstitutionAddress': '6th Street SE 2021,Minneapolis/23c034/,Chicago,US,55455',
 'InstitutionName': 'CMRR 7T/PS',
 'InstitutionalDepartmentName': 'CMRR',
 'MRAcquisitionType': '3D',
 'MagneticFieldStrength': 6.98,
 'Manufacturer': 'Siemens',
 'ManufacturersModelName': 'Investigational_Device_7T',
 'MatrixCoilMode': 'GRAPPA',
 'Modality': 'MR',
 'NonlinearGradientCorrection': False,
 'ParallelReductionFactorInPlane': 3,


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig_rows = 5
fig_cols = 5

n_subplots = fig_rows * fig_cols
n_slice = data.shape[2]
step_size = n_slice // n_subplots
plot_range = n_subplots * step_size
start_stop = int((n_slice - plot_range) / 2)

fig, axs = plt.subplots(fig_rows, fig_cols, figsize=[10, 10])

for idx, img in enumerate(range(start_stop, plot_range, step_size)):
    axs.flat[idx].imshow(data[:, :, img, 2], cmap='gray')
    axs.flat[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
print(np.max(data))

1531.0
